# Analysis of Experiment 3 (Neuroscience)
In this notebook I will walk through the analysis I did for the first Experiment. There are 2 major sections:
* Preperation and First Look
* Results

## Preperation and First Look

In [13]:
#Modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import ttest_ind
from utils_openAI import *

In [14]:
#read data
df = pd.read_csv('NeuroExp3.csv')
df.shape
df.head()

,Unnamed: 0,stimulus_number,condition,good,jargon,neuroscience,answer
0,0,0,Neuro Science and Jargon Good,1,1,1,2
1,0,0,Without Neuro Science Good,1,0,0,2
2,0,0,Without Neuro Science Good,1,0,0,3
3,0,0,Without Neuro Science Good,1,0,0,2
4,0,0,Without Neuro Science Good,1,0,0,2


In [15]:
#get the value count of the different conditions
print(df['condition'].value_counts())
#get value counts of stimulous
print(df['stimulus_number'].value_counts())


Without Neuro Science Good       79
Simple Neuro Science Bad         70
Neuro Science and Jargon Good    69
Simple Neuro Science Good        68
Without Neuro Science Bad        60
Neuro Science and Jargon Bad     54
Name: condition, dtype: int64
0    137
1    135
2    128
Name: stimulus_number, dtype: int64


In [16]:
#only keep columns of interest
df = df[['stimulus_number', 'condition', 'good', 'jargon', 'neuroscience', 'answer']]
#make good, jargon and neuroscience, answer and stimulus number numeric
df['good'] = df['good'].astype(int)
df['jargon'] = df['jargon'].astype(int)
df['neuroscience'] = df['neuroscience'].astype(int)
df['answer'] = df['answer'].astype(int)
df['stimulus_number'] = df['stimulus_number'].astype(int)
#create one hot encoded version of condition
df_ohe = pd.get_dummies(df, columns = ['condition']) #One hot encode conditions in seperate df
df_ohe = pd.get_dummies(df_ohe, columns = ['stimulus_number']) #One hot encode conditions in seperate df
df_ohe.head()

,good,jargon,neuroscience,answer,condition_Neuro Science and Jargon Bad,condition_Neuro Science and Jargon Good,condition_Simple Neuro Science Bad,condition_Simple Neuro Science Good,condition_Without Neuro Science Bad,condition_Without Neuro Science Good,stimulus_number_0,stimulus_number_1,stimulus_number_2
0,1,1,1,2,0,1,0,0,0,0,1,0,0
1,1,0,0,2,0,0,0,0,0,1,1,0,0
2,1,0,0,3,0,0,0,0,0,1,1,0,0
3,1,0,0,2,0,0,0,0,0,1,1,0,0
4,1,0,0,2,0,0,0,0,0,1,1,0,0


## Results

In [17]:
#is rating depending on it being good, jargon, ns
#regression with answer as DV and jargon, good, neuroscience as IV
model = ols('answer ~ jargon  *  neuroscience + good', data = df_ohe).fit(cov_type = 'HC3')
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 answer   R-squared:                       0.431
Model:                            OLS   Adj. R-squared:                  0.426
Method:                 Least Squares   F-statistic:                     228.2
Date:                Tue, 02 May 2023   Prob (F-statistic):           6.13e-86
Time:                        16:45:18   Log-Likelihood:                -770.54
No. Observations:                 400   AIC:                             1549.
Df Residuals:                     396   BIC:                             1565.
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -1.8218    

/Users/tim/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 4, but rank is 3
  warnings.warn('covariance of constraints does not have full '


In [18]:
print(ols('answer ~ jargon', data = df_ohe).fit(cov_type = 'HC3').summary())
print(ols('answer ~ neuroscience', data = df_ohe).fit(cov_type = 'HC3').summary())
print(ols('answer ~ good', data = df_ohe).fit(cov_type = 'HC3').summary())
print(ols('answer ~ jargon  +  neuroscience + good', data = df_ohe).fit(cov_type = 'HC3').summary())

                            OLS Regression Results                            
Dep. Variable:                 answer   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.039
Method:                 Least Squares   F-statistic:                     20.15
Date:                Tue, 02 May 2023   Prob (F-statistic):           9.37e-06
Time:                        16:45:22   Log-Likelihood:                -874.81
No. Observations:                 400   AIC:                             1754.
Df Residuals:                     398   BIC:                             1762.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0650      0.138      0.471      0.6